# Random Forest 4

**Custom Loss Function**  
Das Erstellen einer benutzerdefinierten Verlustfunktion ("Custom Loss Function") in einem Machine-Learning-Modell bedeutet, dass eine spezielle Funktion definiert wird, die das Training des Modells steuert, indem sie Fehler unterschiedlich gewichtet, je nach den Eigenschaften der Daten.

Warum eine Custom Loss Function verwenden?

In unserem Fall, wo die Zielvariable stark unausgewogen ist (viele Nullwerte und wenige Werte nahe 0,2), könnte man eine Verlustfunktion entwickeln, die größere Strafen für Fehler bei den selteneren Zielwerten (also die, die nahe 0,2 liegen) auferlegt. Dadurch wird das Modell stärker auf die genauere Vorhersage dieser selteneren, aber wichtigeren Werte trainiert.

### 1. Anwenden von Random Forest


In [10]:
import pandas as pd
import geopandas as gpd

# GeoJSON-Datei laden
gdf = gpd.read_file('../../data/processed_data/simra_osm_all.geojson')

# Entfernen der Polygone (Geometrie) und nicht benötigter Spalten (type ist ein Feature von SimRa und kann deshlab nicht genutzt werden)
df = gdf.drop(columns=['markers', 'id', 'index_right', 'geometry', 'highway', 'incidents', 'rides', 'type'])


# Anzeigen der ersten Zeilen und Spaltennamen, um zu prüfen, wie die Daten aussehen
df.head(2)

,score,maxspeed,asphalt,concrete,paving_stone,sett,unpaved,cycleway,footway,highway_rare,living_street,path,primary,residential,secondary,service,tertiary,track
0,0.0,50,0.0,1.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.75,0.0,0.0,0.0
1,0.0,30,1.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.0,0.00,0.0,0.0,0.0


In [2]:
# Zielvariable (Score) definieren
y = df['score']

In [3]:
# Alle anderen Spalten als Features verwenden ('score')
X = df.drop(columns=['score'])

In [4]:
from sklearn.model_selection import train_test_split

# Aufteilen der Daten in Trainings- und Testsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Anzahl der Trainingsdaten: {X_train.shape[0]}")
print(f"Anzahl der Testdaten: {X_test.shape[0]}")

Anzahl der Trainingsdaten: 12529
Anzahl der Testdaten: 3133


In [5]:
from sklearn.ensemble import RandomForestRegressor  # Verwende RandomForestRegressor für Regression

# Random Forest Regressor Modell erstellen
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Modell trainieren
rf.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [6]:
y_pred = rf.predict(X_test)


**Ausgabe des MSE und R^2 als Vergleichwert**

In [7]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test-MSE: {mse:.4f}")
print(f"Test-R^2: {r2:.4f}")


Test-MSE: 0.0025
Test-R^2: -0.1238


Standard-MSE (0.0025) zeigt, dass das Modell insgesamt einigermaßen nah an den tatsächlichen Werten liegt, aber dieser Wert wird stark von den vielen Nullwerten beeinflusst.  

Negativer R²-Wert (-0.1238) zeigt, dass das Modell auf dem Testset schlecht generalisiert und dass seine Vorhersagen im Vergleich zu einem einfachen Mittelwertmodell schlecht abschneiden.

### 2. Anwenden der benutzerdefinierten Verlustfunktion

Berechnung nach dem Training und der Standardbewertung die benutzerdefinierte Verlustfunktion auf den Testdaten. Dadurch kann man besser verstehen, wie das Modell bei selteneren Zielwerten abschneidet:

In [9]:
import numpy as np

def custom_weighted_mse(y_true, y_pred):
    mse = (y_true - y_pred) ** 2
    weights = np.where(y_true != 0, 5, 1)
    weighted_mse = weights * mse
    return np.mean(weighted_mse)

custom_mse = custom_weighted_mse(y_test, y_pred)
print(f"Custom Weighted MSE: {custom_mse:.4f}")


Custom Weighted MSE: 0.0115


Höherer Custom Weighted MSE (0.0115) deutet darauf hin, dass das Modell insbesondere bei den seltenen, nicht 0 Zielwerten größere Fehler macht.

#### FAZIT:
Custom Loss Function führt nicht zu besseren Ergebnissen, nur zu einem besseren Vertsändnis unseres Modells...

Warum hat die Custom Loss Function nicht viel gebracht?

- Grundlegende Modellprobleme: Wenn das Modell selbst nicht in der Lage ist, die zugrunde liegenden Muster in den seltenen Werten zu erkennen, kann auch eine Anpassung der Verlustfunktion nur begrenzte Verbesserungen bringen. Möglicherweise sind die Merkmale (Features), die verwendet werden, nicht ausreichend aussagekräftig, um die Unterschiede zwischen den seltenen Zielwerten und den häufigen Nullwerten korrekt zu erfassen.

- Schwierigkeit bei der Unterscheidung: Die Daten sind stark unausgeglichen, mit den meisten Zielwerten bei 0 und wenigen Werten nahe bei 0, aber nicht 0. Das Modell könnte Schwierigkeiten haben, diese seltenen, aber wichtigen Werte zu unterscheiden, besonders wenn die Unterschiede in den Eingangsmerkmalen (Features) subtil sind.

- Modelltyp: Der Random Forest ist ein robustes Modell, aber es hat seine Grenzen, insbesondere wenn es darum geht, komplexe Muster in stark ungleich verteilten Daten zu erkennen. Andere Modelltypen wie Gradient Boosting Machines (z.B. XGBoost, LightGBM) oder neuronale Netze könnten besser geeignet sein.